## Tracking the NASA Satellite

For this assignment, we will be working on streaming data which is satellite location coordinates being provided by NASA.

The data is in the following format:

{"timestamp": 1667492679, "iss_position": {"longitude": "-56.8155", "latitude": "-42.5979"}, "message": "success"}

Data can be accessed from http://api.open-notify.org/iss-now.json

The link can be thought of as the producer as it will do producer job.

Write the consumer to connect and fetch data. In the consumer, you have to write the code which takes the data from producer and uses the location coordinates to plot the satellite location on the world map.

The streaming interval should be 5 seconds. The producer should run for an hour. The graph will show the satellite tracking for 1 hr.

P.S.: Keeping in mind that satellite travels very fast, the graph will have more than half of map covered with satellite track

## Execution

1. It establishes a connection to the ISS location API and continuously updates the ISS position for one hour, collecting latitude and longitude values.

2. These latitude and longitude values are used to plot markers on a custom map created with Folium, displaying the ISS's location in real-time.

3. To enhance the map, each marker is equipped with a tooltip that provides latitude and longitude information when hovered over.

4. The collected latitude and longitude data are simultaneously stored in a Pandas DataFrame and saved to a CSV file for future analysis.

5. Finally, the code generates an HTML map and an interactive display of the map, allowing users to visually track the ISS's journey and access its location data.

In [1]:
#Importing the required libraries
import requests
import time
import folium
from IPython.display import HTML
import pandas as pd  

In [2]:
#Creating an empty DataFrame to store latitude and longitude values
iss_data_df = pd.DataFrame(columns=["Latitude", "Longitude"])

In [3]:
#Creating a Function to fetch data from the API and plot satellite location on a map
def fetch_and_plot_iss_location_for_one_hour():
    
    global iss_data_df  #Declaring the DataFrame as a global variable

    #Creating a Folium map with custom boundaries and zoom level
    iss_map = folium.Map(location=[0, 0], zoom_start=3, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180)

    #Fetching data and plot for 1 hour (3600 seconds)
    end_time = time.time() + 3600
    previous_coords = None
    data_to_append = []

    while time.time() < end_time:
        try:
            #Fetching data from the API
            response = requests.get("http://api.open-notify.org/iss-now.json")
            data = response.json()

            #Extracting latitude and longitude
            latitude = float(data["iss_position"]["latitude"])
            longitude = float(data["iss_position"]["longitude"])

            #Adding a marker for the current ISS location with a tooltip
            folium.Marker(
                location=[latitude, longitude],
                tooltip=f"Lat: {latitude}, Lon: {longitude}"  #Tooltip with latitude and longitude
            ).add_to(iss_map)

            #Connecting the current location to the previous location with a line
            if previous_coords:
                folium.PolyLine([previous_coords, [latitude, longitude]], color="blue").add_to(iss_map)

            previous_coords = [latitude, longitude]

            #Appending the latitude and longitude values to the data_to_append list
            data_to_append.append({"Latitude": latitude, "Longitude": longitude})

            #Sleeping for 5 seconds before fetching the next data
            time.sleep(5)

        except Exception as e:
            print(f"Error: {str(e)}")

    #Concatenatating the data_to_append list into the DataFrame
    iss_data_df = pd.concat([iss_data_df, pd.DataFrame(data_to_append)], ignore_index=True)

    #Saving the map as an HTML file
    iss_map.save("iss_location_tracking.html")

    print("ISS location tracking for 1 hour is plotted on the map. Check iss_location_tracking.html for the map.")


In [4]:
#Running the function to fetch and plot the ISS location for 1 hour
fetch_and_plot_iss_location_for_one_hour()

Error: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
ISS location tracking for 1 hour is plotted on the map. Check iss_location_tracking.html for the map.


In [5]:
#Saving the DataFrame to a CSV file
iss_data_df.to_csv("iss_location_data.csv")

In [6]:
#Displaying the generated HTML map as an image
HTML('<iframe src="iss_location_tracking.html" width="800" height="600"></iframe>')

C:\Users\AKHIL MOTHE\Anaconda3\lib\site-packages\IPython\core\display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
